In [5]:
import torch
from torch import nn, optim
import torch.utils.data as Data
import numpy as np
import torchvision
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt

class YOLO_CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3,  # 输入图片的channel，灰度图像为1，彩色图像为3
                              out_channels=16, # 可以理解为卷积核的个数，也就是输出的channel
                              kernel_size=3,   # 卷积核的大小(3*3)
                              stride=1,        # 步长
                              padding=1)       
        self.conv2 = nn.Conv1d(in_channels=16,
                               out_channels=32,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.conv3 = nn.Conv1d(in_channels=32,
                               out_channels=64,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.fc1 = nn.Linear(960,256)  # 全连接层, 输出为 72 类
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,44)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self,x):
        x = F.max_pool1d(F.relu(self.conv1(x)),2)  # (3,121)-->(16,16,16)-->(16,8,8)
        
        
        x = F.max_pool1d(F.relu(self.conv2(x)),2)  # (16,8,8)-->(32,8,8)-->(32,4,4)
        
        
        x = F.max_pool1d(F.relu(self.conv3(x)),2)  # (32,4,4)-->(64,4,4)-->(64,2,2)
        
        
        x = x.view(x.size(0),-1)                   # 展平为一维数据 (batch_size, 64 * 2 * 2)
        #print(x.shape)
        x = F.relu(self.fc1(x))                    
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.dropout(x)

        return x
    
class YOLO_Loss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, y):

        return torch.sum(torch.pow((x - y), 2))